In [2]:
import numpy as np

In [3]:
# resampling 예시

elements = ['one', 'two', 'three'] 
weights = [0.2, 0.3, 0.5]

from numpy.random import choice
print(choice(elements))
print(choice(elements, size=10, replace=True, p=weights))
# element를 weight 비율로 가져옴.

three
['three' 'one' 'three' 'two' 'three' 'one' 'three' 'one' 'one' 'two']


In [4]:
X = np.load("./tatanic_X_train.npy")
y = np.load("./tatanic_y_train.npy")

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, \
                            test_size=0.3, random_state=101)

In [6]:
X_train[:2]

array([[0.22334962, 0.01517579, 0.        , 1.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ],
       [0.22334962, 0.04489301, 0.11111111, 0.5       , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 1.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

In [7]:
y_train[:10]

array([0., 1., 1., 1., 0., 0., 1., 1., 0., 1.])

In [8]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier


In [9]:
eclf = AdaBoostClassifier\
(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=500, 
                          learning_rate=0.1)

In [10]:
from sklearn.model_selection import cross_val_score
cross_val_score(eclf, X_train, y_train, cv=5).mean()

0.8023041474654379

In [11]:
from sklearn.tree import DecisionTreeClassifier
DecisionTreeClassifier()

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [12]:
AdaBoostClassifier()

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [13]:
params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2,3,4,5],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

# "base_estimator__criterion" : ["gini", "entropy"], => 
#  "base_estimator__max_features" : [7,8,],
#  "base_estimator__max_depth" : [1,2], => 3 이상이면 안됨.
#  ==> base estimator(ensemble model) 속의 parameter (decision tree parameter)
#  "n_estimators": [23,24, 25, 26, 27],
#  "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]


In [14]:
# gridsearch(=>GridSearchCV), cross_validation(=>cv), ensemble(=>eclf) 통합 분석

# gridsearchcv operation
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_train, y_train)

C:\Users\user\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [ ]:
# gridsearch할 때, cpu 갯수 늘려주어야 => n_jobs

In [15]:
# best soore
grid.best_score_

0.8183279742765274

In [16]:
# best parameters
grid.best_params_

{'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 4,
 'base_estimator__max_features': 8,
 'learning_rate': 0.45,
 'n_estimators': 26}

In [17]:
grid.best_estimator_.feature_importances_ # 가장 영향을 많이 준 feature

array([0.34217645, 0.2695191 , 0.05518725, 0.03757004, 0.05232629,
       0.05435819, 0.01648334, 0.01224012, 0.01857194, 0.02115055,
       0.00116594, 0.        , 0.00567116, 0.00994251, 0.00507236,
       0.04370575, 0.00976716, 0.00142078, 0.00547086, 0.        ,
       0.00340828, 0.01280574, 0.0088178 , 0.00779387, 0.00239547,
       0.00241158, 0.00056747])

In [18]:
grid.score(X_train, y_train)

0.9646302250803859

In [19]:
grid.score(X_test, y_test)

0.8014981273408239

In [20]:
preds = grid.predict(X_test)
from sklearn.metrics import confusion_matrix, classification_report

In [21]:
# confusion matrix
confusion_matrix(y_test, preds)

array([[140,  23],
       [ 30,  74]], dtype=int64)

In [22]:
# classification report
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

         0.0       0.82      0.86      0.84       163
         1.0       0.76      0.71      0.74       104

   micro avg       0.80      0.80      0.80       267
   macro avg       0.79      0.79      0.79       267
weighted avg       0.80      0.80      0.80       267

